In [1]:
#!/usr/bin/env python

import sys
import cv2
import os
import numpy as np
import rospy
import message_filters
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

In [2]:
class Image_Converter(object):
    
    def __init__(self, color_topic, depth_topic=None, save_dir=None):
        self.bridge = CvBridge()
        if save_dir:
            self.save_dir = save_dir
        if depth_topic:
            image_sub = message_filters.Subscriber(color_topic, Image)
            depth_sub = message_filters.Subscriber(depth_topic, Image)

            self.timeSync = message_filters.ApproximateTimeSynchronizer(
                            [image_sub, depth_sub], 10, 0.5)
            self.timeSync.registerCallback(self.callback)
        else:
            image_sub = rospy.Subscriber(color_topic, Image, self.callback_single)
        
    def callback(self, image_msg, depth_msg):
        header = image_msg.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, "bgr8")
            depth_data = self.bridge.imgmsg_to_cv2(depth_msg, "passthrough")
        except CvBridgeError as e:
            print(e)
            
        depth_data = np.array(depth_data, dtype=np.uint16)
        
        cv2.imwrite("%s.%s.png" %(secs, nsecs), image_data)
        cv2.imwrite("%s.%s.pgm" %(secs, nsecs), depth_data)
        
    def callback_single(self, image_msg):
        header = image_msg.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, 'bgr8')
        except CvBridgeError as e:
            print(e)
        file_name = os.path.join(self.save_dir, "%s.%s.png" %(secs, nsecs))
        cv2.imwrite(file_name, image_data)
        print('[Info]: %s saved!' % file_name)

In [3]:
class Image_Conv_Visual(Image_Converter):
    
    def __init__(self, color_topic, depth_topic=None, save_dir=None, checkerBoard=None):
        # Inheritance
        super(Image_Conv_Visual, self).__init__(color_topic, depth_topic=depth_topic, save_dir=save_dir)
        # New features.
        self.windows = cv2.namedWindow('Image', 0)
        self.checkerBoard = checkerBoard
        self.criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
    def callback_single(self, image_msg):
        header = image_msg.header
        secs, nsecs = header.stamp.secs, header.stamp.nsecs
        
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, 'bgr8')
        except CvBridgeError as e:
            print(e)
        img = image_data
        gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, self.checkerBoard, None)
        if ret:
            corners = cv2.cornerSubPix(gray, 
                                       corners,
                                       (5,5),
                                       (-1,-1),
                                       self.criteria)
            img = cv2.drawChessboardCorners(img, 
                                                   self.checkerBoard, 
                                                   corners, 
                                                   ret)
        cv2.imshow('Image', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            rospy.signal_shutdown('Exit.')
            print('[End]: Exitting...')
            sys.exit(0)
        if cv2.waitKey(1) & 0xFF == ord('s'):
            file_name = os.path.join(self.save_dir, "%s.%s.png" %(secs, nsecs))
            cv2.imwrite(file_name, image_data)
            print('[Info]: %s saved!' % file_name)
            
    def callback(self, image_msg, depth_msg):
        header = image_msg.header
        secs, nsecs = header.stamp.secs, header.stamp.nsecs
        try:
            image_data = self.bridge.imgmsg_to_cv2(image_msg, 'bgr8')
            depth_data = self.bridge.imgmsg_to_cv2(depth_msg, 'passthrough')
        except CvBridgeError as e:
            print(e)
            
        depth_data = depth_data.astype(np.uint16)
        
        cv2.imshow('Image', image_data)
        k = cv2.waitKey(1)
        if k == ord('q') or k == 27:
            cv2.destroyAllWindows()
            rospy.signal_shutdown('Exit...')
            
        if k == ord('s'):
            image_name = os.path.join(self.save_dir, '%s.%s.png' %(secs, nsecs))
            depth_name = os.path.join(self.save_dir, '%s.%s_.png' %(secs, nsecs))
            cv2.imwrite(image_name, image_data)
            cv2.imwrite(depth_name, depth_data)
            print('[Info]: %s and %s saved!' % (image_name, depth_name))

In [4]:
rospy.init_node('image_converter', anonymous=True)
cap1 = Image_Conv_Visual('/rs01/color/image_rect_color_resize',
                         '/rs01/aligned_depth_to_color/image_raw_resize',
                         save_dir='/media/commaai-03/Data/workdata/temp/', 
                         checkerBoard=None)
try:
    rospy.spin()
except KeyboardInterrupt:
    print("Image Converter Shutting Down.")